# 3D Mesh Analysis
This notebook contains methods for 3D mesh analysis and visualization. It is dependent on the __plotly__ package for data and mesh visualization, and on common scipy packages like __numpy__ for data processing. All geometric features are computed natively without other external dependencies to ensure correctness.


In [1]:
#Setup and check required packages
import plotly
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
print("Plotly: ",plotly.__version__)
import numpy as np
print("NumPy: ",np.__version__)
import matplotlib as mplt
import matplotlib.pyplot as plt
print("Matplotlib: ", mplt.__version__)

import time

Plotly:  3.10.0
NumPy:  1.16.3
Matplotlib:  3.0.3


https://plot.ly/python/

## Action Items:
* ~~Mesh extraction~~
* ~~Mesh visualization~~
* ~~Normal vector calculation, visualization~~
* ~~Mean curvature calculation, visualization~~
* ~~Corrected curvature calculation, zero handling~~
* ~~Directional curvature components~~
* ~~Normal vector histogram~~
* ~~Directional curvature histograms~~
* ~~Ear tip and nose geometric identification~~
* ~~Improved geometric identification robustness~~
* ~~Region selection based on geometric landmarks~~
* ~~Region selection based on normal vector~~
* ~~Bulk processing~~
* Old metrics versus improved metrics
* ~~Metric analysis between groups~~
* Improve by preprocessing head alignment and position

* ~~Object-oriented~~
* ~~Surface area metrics~~
* ~~Debug landmark creation~~
* ~~Results and group results~~

* ~~Split patient load from region/result update~~
* ~~Multiple regions - front half vs back half of cranium~~
* ~~Result - per-vertex ratio of x to total curvature~~
* Debug directional curvature magnitude
* T test
* Analysis - separation of pre vs post for each metric - T test all
* Analysis of best metric between groups
* Plot metrics versus post-op month to see age correlation

Extracted data features:

    vertices
    faces
    minX/maxX/rangeX
    maxrange
    vertexNeighbors
    faceNeighbors
    faceAreas
    faceNormals
    vertexNormals
    vertexElevation
    vertexAzimuth
    vertexCurvatures
    vertexDirectionalCurvatures
    leftEarIndex, rightEarIndex, nasionIndex, noseIndex

In [2]:
%run Multi_Mesh_Definitions.ipynb

Plotly:  3.10.0
NumPy:  1.16.3
Matplotlib:  3.0.3


Done loading Multi_Mesh_Definitions


BEGIN  - Main Code

In [3]:
#Add patients to list - multithreaded version

import os
import re
import multiprocessing
from multiprocessing import Pool

class PatientData ():
    def __init__(self, patientIndex,preOpFile,postOpFile,group,patientNumber,operationType,postOpYears):
        self.patientIndex = patientIndex
        self.preOpFile = preOpFile
        self.postOpFile = postOpFile
        self.group = group
        self.patientNumber = patientNumber
        self.operationType = operationType
        self.postOpYears = postOpYears
        
def createPatientParallel(patientData):
    #print("Starting proc " + str(patientData.patientIndex))
    preOpObjFile = open(patientData.preOpFile,"rt")
    postOpObjFile = open(patientData.postOpFile,"rt")
    preOpMesh = obj_data_to_Mesh(preOpObjFile.read())
    postOpMesh = obj_data_to_Mesh(postOpObjFile.read())
    preOpObjFile.close()
    postOpObjFile.close()
    currPatient=Patient(preOpMesh,postOpMesh,patientData.group,patientData.patientNumber,patientData.operationType,patientData.postOpYears)
    #print("Completing proc " + str(patientData.patientIndex))
    return currPatient

    
startingDir = '/home/dstow/research/med/scans/SickKids'
dirlist = os.listdir(startingDir)
patientList = list()
patientIndex=0
patientDataList = list()

for group in dirlist:
    groupDir = startingDir+'/'+group
    patientDirList = os.listdir(groupDir)
    for patient in patientDirList:
        patientDir = groupDir+'/'+patient
        preOpObjFile=""
        postOpObjFile=""
        patientNumber=""
        operationType=""
        postOpYears=999
        match=re.search("(\d+)_(\D+)$",patient)
        if(match):
            patientNumber=match.group(1)
            operationType=match.group(2)            
        else:
            print("Error: patient number or operation type not found for "+patient)
            break
        opDirList = os.listdir(patientDir)
        for op in opDirList:
            opDir = patientDir+'/'+op
            if(re.search("^Pre",op)):
                fileDirList = os.listdir(opDir)
                simpFound=False
                for file in fileDirList:
                    if(re.search("simp.obj",file)):
                        simpFound=True
                        preOpFile = opDir+'/'+file
                if(not simpFound):
                    print("Error: *simp.obj file not found for directory "+opDir)
            match=re.search("^Post.*(\d+\.\d+)",op)
            if(match):
                postOpYears=match.group(1)
                fileDirList = os.listdir(opDir)
                simpFound=False
                for file in fileDirList:
                    if(re.search("simp.obj",file)):
                        simpFound=True
                        postOpFile = opDir+'/'+file
                if(not simpFound):
                    print("Error: *simp.obj file not found for directory "+opDir)
        print("Creating meshes for patient file "+patientDir)
        patientDataList.append(PatientData(patientIndex,preOpFile,postOpFile,group,patientNumber,operationType,postOpYears))
        patientIndex=patientIndex+1
        
        
#print( "Calculating geometric results for " + str(patientIndex) + " patients." )
print( "Calculating geometric results for " + str(len(patientDataList)) + " patients." )
    

#Launch procs
start_time = time.time()
pool = multiprocessing.Pool(processes=64)
patientList = pool.map(createPatientParallel, patientDataList)

    
end_time = time.time()
total_time = end_time - start_time
print("Done! patientList count:",len(patientList)," time:",str(total_time))

Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps2000_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps1737_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps1849_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps1813_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps1831_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps1804_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps1766_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/Phillips/Ps1760_TCVR
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/Phillips/Ps1388_ESC
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/Phill

In [4]:
# Build resultListPre and resultListPost - ordered lists with pre and post op meshes
# These lists are used for later analysis

resultListPre = list()
resultListPost = list()
for currPatient in patientList:
    currResultPre = Results(currPatient,isPre=True)
    currResultPost = Results(currPatient,isPre=False)
    resultListPre.append(currResultPre)
    resultListPost.append(currResultPost)
    
print("resultListPre count: ",len(resultListPre))
print("resultListPost count: ",len(resultListPost))

resultListPre count:  44
resultListPost count:  44


In [13]:
#for currPatient in patientList:
#    plot_mesh_landmarks(currPatient.preOpMesh)

In [6]:
#First, sort pre and post lists into lists for each operation type
#Next, build GroupResults and DeltaResults to extract group trends
#Finally, perform statistical testing with GroupCompare for each metric
# Only display the significant metrics


endoResultsPre=list()
escResultsPre=list()
tcvrResultsPre=list()
endoResultsPost=list()
escResultsPost=list()
tcvrResultsPost=list()

#print(len(resultListPre))

significance = 0.01


for currResultPre in resultListPre:
    if(currResultPre.operationType=="Endoscopic"):
        endoResultsPre.append(currResultPre)
    elif(currResultPre.operationType=="ESC"):
        escResultsPre.append(currResultPre)
    elif(currResultPre.operationType=="TCVR"):
        tcvrResultsPre.append(currResultPre)
    else:
        print("ERROR: no valid operation type for patient ")

endoGroupResultsPre=GroupResults(endoResultsPre)
escGroupResultsPre=GroupResults(escResultsPre)
tcvrGroupResultsPre=GroupResults(tcvrResultsPre)

#print("ESCPre: ",escGroupResultPre.resultCount)
#print("TCVRPre: ",tcvrGroupResultPre.resultCount)

#print(len(resultListPost))

for currResultPost in resultListPost:
    if(currResultPost.operationType=="Endoscopic"):
        endoResultsPost.append(currResultPost)
    elif(currResultPost.operationType=="ESC"):
        escResultsPost.append(currResultPost)
    elif(currResultPost.operationType=="TCVR"):
        tcvrResultsPost.append(currResultPost)
    else:
        print("ERROR: no valid operation type for patient ")

endoGroupResultsPost=GroupResults(endoResultsPost)
escGroupResultsPost=GroupResults(escResultsPost)
tcvrGroupResultsPost=GroupResults(tcvrResultsPost)

endoDeltaResults = DeltaResults(endoResultsPre,endoResultsPost)
escDeltaResults = DeltaResults(escResultsPre,escResultsPost)
tcvrDeltaResults = DeltaResults(tcvrResultsPre,tcvrResultsPost)

print("Endo-ESC")
for currMetric in endoGroupResultsPost.metrics:
    gc=GroupCompare(endoGroupResultsPost,escGroupResultsPost,currMetric)
    gcDelta=GroupCompare(endoDeltaResults,escDeltaResults,currMetric)
    gcPre=GroupCompare(endoGroupResultsPre,escGroupResultsPre,currMetric)
    #if(gc.p<significance):
    #    print("Post Metric:",gc.metric," p:",gc.p, " change:",gc.x2-gc.x1)
    #if(gcDelta.p<significance):
    #    print("Delta Metric:",gc.metric," p:",gc.p)
    #print(gcDelta.metric,",",gcDelta.p)
    if(gcPre.p<significance):
        print("Pre Metric:",gcPre.metric," p:",gcPre.p, " change:",gcPre.x2-gcPre.x1)
    
print("Endo-TCVR")
for currMetric in endoGroupResultsPost.metrics:
    gc=GroupCompare(endoGroupResultsPost,tcvrGroupResultsPost,currMetric)
    gcDelta=GroupCompare(endoDeltaResults,tcvrDeltaResults,currMetric)
    gcPre=GroupCompare(endoGroupResultsPre,tcvrGroupResultsPre,currMetric)
    #if(gc.p<significance):
    #    print("Post Metric:",gc.metric," p:",gc.p, " change:",gc.x2-gc.x1)
    #if(gcDelta.p<significance):
    #    print("Delta Metric:",gc.metric," p:",gc.p)
    #print(gcDelta.metric,",",gcDelta.p)
    if(gcPre.p<significance):
        print("Pre Metric:",gcPre.metric," p:",gcPre.p, " change:",gcPre.x2-gcPre.x1)
   
    
print("ESC-TCVR")
for currMetric in endoGroupResultsPost.metrics:
    gc=GroupCompare(escGroupResultsPost,tcvrGroupResultsPost,currMetric)
    gcDelta=GroupCompare(escDeltaResults,tcvrDeltaResults,currMetric)
    gcPre=GroupCompare(escGroupResultsPre,tcvrGroupResultsPre,currMetric)
    #if(gc.p<significance):
    #    print("Post Metric:",gc.metric," p:",gc.p, " change:",gc.x2-gc.x1)
    #if(gcDelta.p<significance):
    #    print("Delta Metric:",gc.metric," p:",gc.p)
    #print(gcDelta.metric,",",gcDelta.p)
    if(gcPre.p<significance):
        print("Pre Metric:",gcPre.metric," p:",gcPre.p, " change:",gcPre.x2-gcPre.x1)
    

        
fig=go.Figure()
fig.add_trace(go.Scatter(x=endoGroupResultsPost.results["curvYPercentBack"],y=endoGroupResultsPost.results["curvZPercentBack"],mode='markers',name='Endo'))
fig.add_trace(go.Scatter(x=escGroupResultsPost.results["curvYPercentBack"],y=escGroupResultsPost.results["curvZPercentBack"],mode='markers',name='ESC'))
fig.add_trace(go.Scatter(x=tcvrGroupResultsPost.results["curvYPercentBack"],y=tcvrGroupResultsPost.results["curvZPercentBack"],mode='markers',name='TCVR'))
fig.show()




print("Endo Delta curvYPercentBack:",endoDeltaResults.means["curvYPercentBack"])
print("ESC Delta curvYPercentBack:",escDeltaResults.means["curvYPercentBack"])
print("TCVR Delta curvYPercentBack:",tcvrDeltaResults.means["curvYPercentBack"])

Endo-ESC
Pre Metric: curvXPercentFront  p: 0.006353999948763822  change: -0.02338673159524679
Pre Metric: curvYPercentFront  p: 0.0052657254779778  change: 0.030025946489332045
Pre Metric: curvYPercentBack  p: 0.002315902949840066  change: -0.022152142308770495
Pre Metric: curvZPercentBack  p: 0.004577833469558159  change: 0.01959870347911913
Endo-TCVR
Pre Metric: curvXPercentFront  p: 0.006286623036868507  change: -0.02565252851957267
Pre Metric: curvYPercentBack  p: 0.0011016101385847164  change: -0.027283480879126965
Pre Metric: curvZPercentBack  p: 0.0006438986628037467  change: 0.02646378153786566
ESC-TCVR
Pre Metric: curvZPercentFront  p: 0.00028787314174848455  change: 0.01975110023174853


Endo Delta curvYPercentBack: -0.029000966596150275
ESC Delta curvYPercentBack: -0.015705125563499085
TCVR Delta curvYPercentBack: -0.015866492057567724


In [7]:
print(len(resultListPre))
print(len(resultListPost))

significance = 0.05

groupResultsPre=GroupResults(resultListPre)
groupResultsPost=GroupResults(resultListPost)
deltaResults=DeltaResults(resultListPre,resultListPost)

for currMetric in groupResultsPre.metrics:
    gc=GroupCompare(groupResultsPre,groupResultsPost,currMetric)
    #if(gc.p<significance):
    #    print("Metric:",gc.metric," p:",gc.p)
    print(gc.metric,",",gc.p)
    
fig=go.Figure()
fig.add_trace(go.Scatter(x=groupResultsPre.results["curvYPercentBack"],y=groupResultsPre.results["curvYPercentBack"],mode='markers',name='Pre'))
fig.add_trace(go.Scatter(x=groupResultsPost.results["curvYPercentBack"],y=groupResultsPost.results["curvYPercentBack"],mode='markers',name='Post'))
fig.show()

plot_hist(groupResultsPre.results["curvYPercentBack"],"Pre-curvYPercentBack")
plot_hist(groupResultsPost.results["curvYPercentBack"],"Post-curvYPercentBack")
plot_hist_double(groupResultsPre.results["curvYPercentBack"],"Pre-curvYPercentBack",groupResultsPost.results["curvYPercentBack"],"Post-curvYPercentBack")

for currMetric in deltaResults.metrics:
    print(currMetric,deltaResults.means[currMetric],deltaResults.stds[currMetric])

44
44
curvFront , 0.1312407268106105
curvXFront , 0.3912365835486379
curvYFront , 0.009853168161566044
curvZFront , 0.44572930296031466
curvXPercentFront , 0.4143703696221708
curvYPercentFront , 0.06552886415362262
curvZPercentFront , 0.0006448756896514284
curvBack , 9.951597663353359e-05
curvXBack , 0.000594080993213901
curvYBack , 1.0471673026620615e-05
curvZBack , 0.00011056940225362593
curvXPercentBack , 0.04047092816041541
curvYPercentBack , 3.0232294964463774e-06
curvZPercentBack , 0.15533205446804865


/home/dstow/.local/lib/python3.6/site-packages/plotly/matplotlylib/renderer.py:487: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.



curvFront -0.0020852845959806313 0.008182302701232911
curvXFront -0.0004931099248177323 0.00340843525551994
curvYFront -0.0013321700371561216 0.002961233479580124
curvZFront -0.00047239283088039934 0.0037874289150068696
curvXPercentFront 0.003492931543235413 0.02288029979373713
curvYPercentFront -0.009237474200078756 0.03249864836985477
curvZPercentFront 0.01248992231981625 0.02378696204424603
curvBack -0.004099231509563466 0.0065657124301686355
curvXBack -0.001451211912085478 0.002685758647087778
curvYBack -0.0018613303272464652 0.002586818820173994
curvZBack -0.0017194157502959174 0.002690831053273597
curvXPercentBack 0.00684238096499127 0.01861600281091262
curvYPercentBack -0.017868049464668415 0.02068261616723124
curvZPercentBack 0.0045686231263215105 0.017735053935890782


In [3]:
#Loading example for reference. Disabled.

if(True):

    #Extract data from 3D mesh vertices

    #testExamples
    #Load file and check
    init_notebook_mode(connected=True)
    #objFileName = "test_robyn_simp.obj"
    objFileName = "test_simp.obj"
    objFile = open(objFileName,"rt")
    testmesh = obj_data_to_Mesh(objFile.read())
    print("Vertices: ",testmesh.vertices.shape)
    print("Faces: ",testmesh.faces.shape)


NameError: name 'obj_data_to_Mesh' is not defined

In [9]:
#Display result examples for reference. Disabled.

if(True):

    plot_hist(np.log(testmesh.vertexDirectionalCurvatures[:,0])+0.0000000005,"Log(X-Curvature)")
    plot_hist(np.log(testmesh.vertexDirectionalCurvatures[:,1])+0.0000000005,"Log(Y-Curvature)")
    plot_hist(np.log(testmesh.vertexDirectionalCurvatures[:,2])+0.0000000005,"Log(Z-Curvature)")

    #Plot with color based on vertex position
    #Prepare color values based on x/y/z coordinate
    vertexColors=np.ones_like(testmesh.vertices)
    np.copyto(vertexColors,testmesh.vertices)
    vertexColors[:,0]-=testmesh.minX
    vertexColors[:,0]/=testmesh.rangeX
    vertexColors[:,1]-=testmesh.minY
    vertexColors[:,1]/=testmesh.rangeY
    vertexColors[:,2]-=testmesh.minZ
    vertexColors[:,2]/=testmesh.rangeZ

    plot_mesh_custom(testmesh,vertexColors,"Vertex Position")
    #Plot with color based on normal vector direction
    #Add one and divide by two to map from [-1,1] to [0,1]
    vertexColors=(testmesh.vertexNormals+1.0)/2.0
    plot_mesh_custom(testmesh,vertexColors,"Vertex Normals")
    plot_mesh_colorscale(testmesh,testmesh.vertexElevation,"Vertex Elevation","Elevation (Degrees)")
    plot_mesh_colorscale(testmesh,testmesh.vertexAzimuth,"Vertex Azimuth","Azimuth (Degrees)")
    #Plot with color based on curvature magnitude, log scale
    plot_mesh_colorscale(testmesh,np.log(testmesh.vertexCurvatures),"Vertex Curvature (Log)","Log(Curvature)")

    #Plot with color based on each direction curvature magnitude, log scale

    plot_mesh_colorscale(testmesh,np.log(testmesh.vertexDirectionalCurvatures[:,0]),"Vertex X-Curvature (Log)","Log(X-Curvature)")
    plot_mesh_colorscale(testmesh,np.log(testmesh.vertexDirectionalCurvatures[:,1]),"Vertex Y-Curvature (Log)","Log(Y-Curvature)")
    plot_mesh_colorscale(testmesh,np.log(testmesh.vertexDirectionalCurvatures[:,2]),"Vertex Z-Curvature (Log)","Log(Z-Curvature)")

Last Updated: 9/2/19